## PE file Preprocessing Pipeline

---
---

STAGE-1
---

---

In [ ]:
# Install required packages
!pip install lief pillow numpy pandas scikit-learn matplotlib pennylane

# Import necessary libraries
import lief
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import json
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

print("All dependencies installed and imported successfully!")

## Step-1: PE file Parsing using LIEF
---


In [ ]:
def parse_pe_file(pe_file_path):
    """
    Parse PE file using LIEF and extract basic information
    """
    print(f"Parsing PE file: {pe_file_path}")
    
    try:
        binary = lief.PE.parse(pe_file_path)
        if not binary:
            print("Failed to parse PE file")
            return None
            
        print(f"Successfully parsed PE file")
        print(f"Number of sections: {len(binary.sections)}")
        print(f"Entry point: 0x{binary.optional_header.addressof_entrypoint:x}")
        
        return binary
        
    except Exception as e:
        print(f"LIEF parsing error: {e}")
        return None

# Test the function
# binary = parse_pe_file("malware_sample.exe")

## Step-2: Target Section Identification
---


In [ ]:
def extract_target_sections(binary):
    """
    Extract the five target sections from PE file
    """
    # Define the five target sections from the paper
    target_sections = [".text", ".data", ".rdata", ".rsrc", ".reloc"]
    section_data = {}
    
    # Extract each target section
    for target_section in target_sections:
        section_found = False
        
        for section in binary.sections:
            # Clean section name (remove null bytes)
            section_name = section.name.strip('\x00')
            
            if section_name == target_section:
                # Extract section content as bytes
                content = section.content
                section_data[target_section] = content
                section_found = True
                
                print(f"Found {target_section}: {len(content)} bytes")
                break
        
        # Handle missing sections (assign -1 score as per paper)
        if not section_found:
            section_data[target_section] = None
            print(f"Section {target_section} not found - will receive -1 score")
    
    return section_data

# Test the function
# section_data = extract_target_sections(binary)

## Step-3: Conversion of Section bytes to 8x8 Grayscale Image
---


In [ ]:
def bytes_to_8x8_grayscale_image(byte_content, section_name):
    """
    Convert section bytes to 8x8 grayscale image using Nataraj method
    """
    if byte_content is None:
        # Missing section - return None
        return None
    
    # Target: 8x8 = 64 bytes needed
    target_pixels = 64
    
    if len(byte_content) >= target_pixels:
        # Take first 64 bytes
        pixel_values = list(byte_content[:target_pixels])
    else:
        # Pad with zeros if insufficient bytes
        pixel_values = list(byte_content) + [0] * (target_pixels - len(byte_content))
    
    # Convert to numpy array and reshape to 8x8
    image_array = np.array(pixel_values, dtype=np.uint8).reshape(8, 8)
    
    # Create PIL Image for visualization/saving
    pil_image = Image.fromarray(image_array, mode='L')  # 'L' for grayscale
    
    return image_array, pil_image

# Test the function
# img_array, pil_img = bytes_to_8x8_grayscale_image(section_data['.text'], '.text'

## Step-4: Processing of each sections grayscale image
---


In [ ]:
def process_sections_to_images(section_data, save_images=False, output_dir="./section_images/"):
    """
    Process all sections to 8x8 grayscale images
    """
    section_images = {}
    
    if save_images:
        os.makedirs(output_dir, exist_ok=True)
    
    for section_name, content in section_data.items():
        if content is not None:
            result = bytes_to_8x8_grayscale_image(content, section_name)
            if result is not None:
                img_array, pil_img = result
                section_images[section_name] = {
                    'array': img_array,
                    'pil': pil_img,
                    'size': len(content),
                    'status': 'found'
                }
                
                # Save image for visualization
                if save_images:
                    filename = f"{section_name.replace('.', '')}_section.png"
                    filepath = os.path.join(output_dir, filename)
                    pil_img.save(filepath)
                    print(f"Saved {section_name} as 8x8 grayscale image: {filepath}")
                else:
                    print(f"Processed {section_name} to 8x8 grayscale image")
        else:
            section_images[section_name] = {
                'array': None,
                'pil': None,
                'size': 0,
                'status': 'missing',
                'score': -1
            }
            print(f"{section_name}: Missing section")
    
    return section_images

# Test the function
# section_images = process_sections_to_images(section_data, save_images=True)

## Step-5: Understanding each Sections Significance
---


In [ ]:
def explain_section_content(section_name, content_size):
    """
    Explain what each PE section typically contains
    """
    explanations = {
        ".text": f"Executable code and CPU instructions ({content_size} bytes)",
        ".data": f"Initialized global and static variables ({content_size} bytes)",
        ".rdata": f"Read-only data like string literals and constants ({content_size} bytes)",
        ".rsrc": f"Resources like icons, menus, strings - frequently exploited by malware ({content_size} bytes)",
        ".reloc": f"Relocation information for loading at different memory addresses ({content_size} bytes)"
    }

    return explanations.get(section_name, f"Unknown section ({content_size} bytes)")

# Display section information
for section_name, section_info in section_images.items():
    if section_info is not None:
        print(f"{section_name}: {explain_section_content(section_name, section_info['size'])}")
    else:
        print(f"{section_name}: Missing section - will receive -1 score")


## Step-6: Complete PE Processing function
---

In [ ]:
def complete_stage1_processing(pe_file_path, save_images=False, output_dir="./section_images/"):
    """
    Complete Stage-1 pipeline: PE file → Section extraction → 8x8 grayscale images
    """
    print(f"=== Stage-1 Processing: {pe_file_path} ===")
    
    # Step 1: Parse PE file
    binary = parse_pe_file(pe_file_path)
    if binary is None:
        return None
    
    # Step 2: Extract target sections
    section_data = extract_target_sections(binary)
    
    # Step 3: Convert sections to 8x8 images
    section_images = process_sections_to_images(section_data, save_images, output_dir)
    
    print("Stage-1 processing completed successfully!")
    return section_images

# Test the complete Stage-1 pipeline
# stage1_results = complete_stage1_processing("malware_sample.exe", save_images=True)

# Stage-2:

---
---

## Step-7: Prepare Section Data for PCA30
---

In [ ]:
def prepare_section_data_for_pca30(dataset_results, section_name):
    """
    Prepare 8x8 section images for PCA30 training
    """
    section_data = []
    missing_indices = []
    
    for i, pe_results in enumerate(dataset_results):
        if pe_results[section_name]['status'] == 'found':
            # Get 8x8 image and flatten to 64 features
            img_8x8 = pe_results[section_name]['array']  # Shape: (8, 8)
            flattened = img_8x8.flatten()  # Shape: (64,)
            section_data.append(flattened)
        else:
            # Track missing sections for later handling
            missing_indices.append(i)
            # Don't add to training data - will handle separately
    
    return np.array(section_data), missing_indices

# Test the function
# valid_data, missing_idx = prepare_section_data_for_pca30([stage1_results], '.text')

# Step-8: Train PCA30 Models
---

In [ ]:
def train_pca30_models(training_dataset_results, n_components=30):
    """
    Train PCA30 models for each section separately
    """
    target_sections = [".text", ".data", ".rdata", ".rsrc", ".reloc"]
    pca_models = {}
    pca_explained_variance = {}
    
    print("Training PCA30 models for each section...")
    
    for section in target_sections:
        print(f"\nTraining PCA30 for {section} section...")
        
        # Get valid samples (exclude missing sections)
        valid_data, missing_idx = prepare_section_data_for_pca30(training_dataset_results, section)
        
        if len(valid_data) > 0:
            # Initialize and fit PCA30: 64 features → 30 components
            pca_model = PCA(n_components=n_components, random_state=42)
            pca_model.fit(valid_data)  # Input: (n_valid_samples, 64)
            pca_models[section] = pca_model
            
            # Track explained variance
            variance_ratios = pca_model.explained_variance_ratio_
            total_variance = np.sum(variance_ratios)
            pca_explained_variance[section] = {
                'individual': variance_ratios,
                'total': total_variance,
                'n_samples': len(valid_data)
            }
            
            print(f"{section}: {total_variance:.3f} total variance explained")
            print(f"Top 5 components: {variance_ratios[:5]}")
            print(f"Training samples: {len(valid_data)}")
            
            # Quality check
            if total_variance < 0.85:  # Expect >85% for 30 components
                print(f"WARNING: {section} PCA30 only retains {total_variance:.1%} variance")
        else:
            print(f"WARNING: No valid samples found for {section} section!")
            pca_models[section] = None
    
    return pca_models, pca_explained_variance

# Test the function
# pca_models, pca_variance = train_pca30_models([stage1_results])